<h3>Simulação de Catálogo de Dados (Ambiente Local)
<h5>Em ambiente local, onde o AWS Glue não está disponível na versão Community do LocalStack, foi utilizada uma abordagem alternativa para validação do catálogo de dados.
    
Utilizando Spark SQL, foi criado um database e uma tabela externa apontando para os dados da camada Analytics armazenados no S3 (LocalStack).

Essa abordagem permitiu:

 * Validação de schema

 * Validação de partições

 * Execução de queries SQL equivalentes ao Amazon Athena

Em ambiente AWS real, essa mesma estrutura seria catalogada automaticamente pelo AWS Glue Crawler.

In [ ]:
# Import do projeto
import sys
import os

PROJECT_ROOT = "/repositorio/projeto"

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

In [ ]:
# Imports
from src.utils.spark_session import criar_spark_session
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from src.utils.spark_session import criar_spark_session
from src.config.settings import settings

# Criar sessão do Spark
spark = criar_spark_session()

# Reduzir ruídos
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# Ler a tabela de Analytics
spark.read.parquet(
    "s3a://data-lake-local/analytics/clientes/"
).show(5)

In [ ]:
# Criação do banco de dados
spark.sql("CREATE DATABASE IF NOT EXISTS clientes")

# Visualizar o banco de dados
spark.sql("SHOW DATABASES").show()


In [ ]:
# Criação da tabela externa (Analytics)
spark.sql("""
    CREATE TABLE IF NOT EXISTS clientes.clientes_analytics
    USING PARQUET
    LOCATION 's3a://data-lake-local/analytics/clientes/'
""")

spark.sql("SHOW TABLES IN clientes").show()

In [ ]:
# Validar Schema e Partiçao
spark.sql("""
    DESCRIBE TABLE clientes.clientes_analytics
""").show(truncate=False)

In [ ]:
# Atualiza o metastore com as partições
spark.sql("""
    MSCK REPAIR TABLE clientes.clientes_analytics
""")

In [ ]:
# Query de validação (Athena) - Total de clientes e média de idade por estado
spark.sql("""
SELECT
    estado,
    COUNT(*) AS total_clientes,
    ROUND(AVG(idade),1) AS idade_media
FROM (
    SELECT
        CASE
            WHEN estado = 'SPP' THEN 'SP'
            ELSE estado
        END AS estado,
        idade
    FROM clientes.clientes_analytics
)
GROUP BY estado
ORDER BY total_clientes DESC
""").show()